# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5e6a0c9820>
├── 'a' --> tensor([[ 0.2350,  0.5643,  0.4092],
│                   [ 0.9333, -0.5676,  2.7982]])
└── 'x' --> <FastTreeValue 0x7f5e6a0c94f0>
    └── 'c' --> tensor([[-1.3771,  1.4400,  1.3595,  0.8534],
                        [ 2.1254,  0.9136, -1.6880,  0.3345],
                        [-0.3717, -0.2446, -1.1421, -0.3759]])

In [4]:
t.a

tensor([[ 0.2350,  0.5643,  0.4092],
        [ 0.9333, -0.5676,  2.7982]])

In [5]:
%timeit t.a

63 ns ± 0.163 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5e6a0c9820>
├── 'a' --> tensor([[ 0.1983, -0.5913,  0.8801],
│                   [ 3.1057,  0.1991, -0.5828]])
└── 'x' --> <FastTreeValue 0x7f5e6a0c94f0>
    └── 'c' --> tensor([[-1.3771,  1.4400,  1.3595,  0.8534],
                        [ 2.1254,  0.9136, -1.6880,  0.3345],
                        [-0.3717, -0.2446, -1.1421, -0.3759]])

In [7]:
%timeit t.a = new_value

62 ns ± 0.0665 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2350,  0.5643,  0.4092],
               [ 0.9333, -0.5676,  2.7982]]),
    x: Batch(
           c: tensor([[-1.3771,  1.4400,  1.3595,  0.8534],
                      [ 2.1254,  0.9136, -1.6880,  0.3345],
                      [-0.3717, -0.2446, -1.1421, -0.3759]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2350,  0.5643,  0.4092],
        [ 0.9333, -0.5676,  2.7982]])

In [11]:
%timeit b.a

53.7 ns ± 0.0246 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4433, -0.3284,  0.8681],
               [ 1.1095,  0.9760, -0.6313]]),
    x: Batch(
           c: tensor([[-1.3771,  1.4400,  1.3595,  0.8534],
                      [ 2.1254,  0.9136, -1.6880,  0.3345],
                      [-0.3717, -0.2446, -1.1421, -0.3759]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.0988 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

837 ns ± 0.0946 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 22.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 441 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5dbacbcbb0>
├── 'a' --> tensor([[[ 0.2350,  0.5643,  0.4092],
│                    [ 0.9333, -0.5676,  2.7982]],
│           
│                   [[ 0.2350,  0.5643,  0.4092],
│                    [ 0.9333, -0.5676,  2.7982]],
│           
│                   [[ 0.2350,  0.5643,  0.4092],
│                    [ 0.9333, -0.5676,  2.7982]],
│           
│                   [[ 0.2350,  0.5643,  0.4092],
│                    [ 0.9333, -0.5676,  2.7982]],
│           
│                   [[ 0.2350,  0.5643,  0.4092],
│                    [ 0.9333, -0.5676,  2.7982]],
│           
│                   [[ 0.2350,  0.5643,  0.4092],
│                    [ 0.9333, -0.5676,  2.7982]],
│           
│                   [[ 0.2350,  0.5643,  0.4092],
│                    [ 0.9333, -0.5676,  2.7982]],
│           
│                   [[ 0.2350,  0.5643,  0.4092],
│                    [ 0.9333, -0.5676,  2.7982]]])
└── 'x' --> <FastTreeValue 0x7f5e60fc7190>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.3 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5dbacdff40>
├── 'a' --> tensor([[ 0.2350,  0.5643,  0.4092],
│                   [ 0.9333, -0.5676,  2.7982],
│                   [ 0.2350,  0.5643,  0.4092],
│                   [ 0.9333, -0.5676,  2.7982],
│                   [ 0.2350,  0.5643,  0.4092],
│                   [ 0.9333, -0.5676,  2.7982],
│                   [ 0.2350,  0.5643,  0.4092],
│                   [ 0.9333, -0.5676,  2.7982],
│                   [ 0.2350,  0.5643,  0.4092],
│                   [ 0.9333, -0.5676,  2.7982],
│                   [ 0.2350,  0.5643,  0.4092],
│                   [ 0.9333, -0.5676,  2.7982],
│                   [ 0.2350,  0.5643,  0.4092],
│                   [ 0.9333, -0.5676,  2.7982],
│                   [ 0.2350,  0.5643,  0.4092],
│                   [ 0.9333, -0.5676,  2.7982]])
└── 'x' --> <FastTreeValue 0x7f5e3c4c5d00>
    └── 'c' --> tensor([[-1.3771,  1.4400,  1.3595,  0.8534],
                        [ 2.1254,  0.9136, -1.6880,  0.3345],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 49.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.5 µs ± 154 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.3771,  1.4400,  1.3595,  0.8534],
                       [ 2.1254,  0.9136, -1.6880,  0.3345],
                       [-0.3717, -0.2446, -1.1421, -0.3759]],
              
                      [[-1.3771,  1.4400,  1.3595,  0.8534],
                       [ 2.1254,  0.9136, -1.6880,  0.3345],
                       [-0.3717, -0.2446, -1.1421, -0.3759]],
              
                      [[-1.3771,  1.4400,  1.3595,  0.8534],
                       [ 2.1254,  0.9136, -1.6880,  0.3345],
                       [-0.3717, -0.2446, -1.1421, -0.3759]],
              
                      [[-1.3771,  1.4400,  1.3595,  0.8534],
                       [ 2.1254,  0.9136, -1.6880,  0.3345],
                       [-0.3717, -0.2446, -1.1421, -0.3759]],
              
                      [[-1.3771,  1.4400,  1.3595,  0.8534],
                       [ 2.1254,  0.9136, -1.6880,  0.3345],
                       [-0.3717, -0.2446, -1.1421, -0.3759]],

In [26]:
%timeit Batch.stack(batches)

105 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.3771,  1.4400,  1.3595,  0.8534],
                      [ 2.1254,  0.9136, -1.6880,  0.3345],
                      [-0.3717, -0.2446, -1.1421, -0.3759],
                      [-1.3771,  1.4400,  1.3595,  0.8534],
                      [ 2.1254,  0.9136, -1.6880,  0.3345],
                      [-0.3717, -0.2446, -1.1421, -0.3759],
                      [-1.3771,  1.4400,  1.3595,  0.8534],
                      [ 2.1254,  0.9136, -1.6880,  0.3345],
                      [-0.3717, -0.2446, -1.1421, -0.3759],
                      [-1.3771,  1.4400,  1.3595,  0.8534],
                      [ 2.1254,  0.9136, -1.6880,  0.3345],
                      [-0.3717, -0.2446, -1.1421, -0.3759],
                      [-1.3771,  1.4400,  1.3595,  0.8534],
                      [ 2.1254,  0.9136, -1.6880,  0.3345],
                      [-0.3717, -0.2446, -1.1421, -0.3759],
                      [-1.3771,  1.4400,  1.3595,  0.8534],
                   

In [28]:
%timeit Batch.cat(batches)

284 µs ± 2.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 699 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
